In [1]:
%load_ext autoreload
%autoreload 2

from scribbles_creator import *
from PIL import Image
import napari
import numpy as np
import os

## Create scribble annotation from an image mask of the cell pose dataset

In [2]:
def create_cellpose_scribble(folder_path, img_num, max_perc=0.2, sq_scaling=False, mode="all", save_res=False, show_res=False):

    # Read the ground truth as an image
    image_path = f"{folder_path}/{str(img_num).zfill(3)}"
    ground_truth = np.array(Image.open(image_path + "_masks.png"))
    # Summarize all non-background classes into one class (we are testing semantic segmentation not instance segmentation)
    ground_truth[ground_truth>0] = 2
    # Set the background class to 1 (since the scribble annotation assumes class 0 to represent non-annotated pixels)
    ground_truth[ground_truth==0] = 1

    scribbles = create_even_scribble(ground_truth, max_perc=max_perc, sq_scaling=False, mode=mode)
    perc_labelled = np.sum(scribbles>0) / (scribbles.shape[0] * scribbles.shape[1]) * 100

    if save_res:
        # Save the scribble annotation as an image
        scribble_img = Image.fromarray(scribbles)
        scribble_img.save(image_path + f"_scribbles_{mode}_{max_perc}.png")
        # If the ground truth file does not exist, save it as well
        gt_file = image_path + "_ground_truth.png"
        if not os.path.exists(gt_file):
            gt_img = Image.fromarray(ground_truth)
            gt_img.save(gt_file)

    if show_res:
        # Show the ground truth and the scribble annotation
        v = napari.Viewer()
        v.add_image(image)
        v.add_labels(ground_truth)
        v.add_labels(scribbles)

    return scribbles, perc_labelled

In [17]:
folder_path = "./imgs/train"
percentages = {}
for bin in [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1]:
    percentages[bin] = []
    for img_num in range(0, 5):
        scribbles, perc_labelled = create_cellpose_scribble(folder_path, img_num, max_perc=bin, sq_scaling=False, mode = "all", show_res=False, save_res=True)
        percentages[bin].append(perc_labelled)

In [18]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.01: mean = 0.0083, std = 0.0005, range = 0.0076 - 0.0087
labelled percentage in bin 0.025: mean = 0.0194, std = 0.0012, range = 0.0173 - 0.0209
labelled percentage in bin 0.05: mean = 0.0405, std = 0.0039, range = 0.0352 - 0.0464
labelled percentage in bin 0.1: mean = 0.0908, std = 0.0025, range = 0.0867 - 0.0933
labelled percentage in bin 0.25: mean = 0.2347, std = 0.0016, range = 0.2320 - 0.2366
labelled percentage in bin 0.5: mean = 0.4838, std = 0.0060, range = 0.4778 - 0.4926
labelled percentage in bin 1: mean = 0.9709, std = 0.0049, range = 0.9669 - 0.9781


In [ ]:
v = napari.Viewer()
v.add_labels(scribbles)

## Load and show scribbles and ground truth

In [19]:
# Show the image, ground truth and the scribble annotation in a napari viewer
img_num = 0
mode = "all"
max_perc = 0.5
folder_path = "./imgs/train"
image_base = f"{folder_path}/{str(img_num).zfill(3)}"
image_path = image_base + "_img.png"
ground_truth_path = image_base + "_ground_truth.png"
scribbles_path = image_base + f"_scribbles_{mode}_{max_perc}.png"
image = np.array(Image.open(image_path))
ground_truth = np.array(Image.open(ground_truth_path))
scribbles = np.array(Image.open(scribbles_path))
v = napari.Viewer()
v.add_image(image)
v.add_labels(ground_truth)
v.add_labels(scribbles)

<Labels layer 'scribbles' at 0x2f10127fcd0>

## When we have performed segmentation in the Napari plugin, we can calculate its accuracy

In [ ]:
segmentation = v.layers['segmentation'].data
acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
print(f"Accuracy (pixels correctly segmented): {acc}%")